In [1]:
# importem les biblioteques per treballar i el dataset prèviament netejat
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import xgboost as xgb
import sklearn.metrics as metrics
import numpy as np

In [2]:
data_gym = pd.read_excel('DatosParaModelo.xlsx')

In [3]:
Registros = len(data_gym)
Registros_Baja = data_gym['Baja'].sum()
Proporcion_Registros_Bajas = Registros_Baja/Registros*100

print(f'La proporción de bajas por registro es de un {Proporcion_Registros_Bajas:.2f}%')

La proporción de bajas por registro es de un 4.60%


# Logistic Regression

In [7]:
def Logistic_Regression (Fechas):

    lista_AUC = []

    Modelo_reg_log = LogisticRegression(penalty=None, max_iter=3000)

    for date in Fechas:
        Train_data = data_gym[data_gym['Mes'] <= date]
        Fecha_test = pd.Timestamp(date) + pd.DateOffset(months=1)
        Test_data = data_gym[data_gym['Mes'] == Fecha_test]
        
        X_train = Train_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
        y_train = Train_data['Baja']
        X_test = Test_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
        y_test = Test_data['Baja']

        Modelo_reg_log.fit(X_train, y_train)
        Predicciones_reg = Modelo_reg_log.predict_proba(X_test)
        AUC_PR = average_precision_score(y_test, Predicciones_reg[:, 1])
        lista_AUC.append(AUC_PR)

    Mean_AUC = np.mean(lista_AUC)
    print(f'El AUC del Modelo de Regresión Logística es: {Mean_AUC}')


# Random Forest

In [8]:
def Random_Forest(Fechas, Profundidades, Estimadores):

  lista_parametros = []
  lista_AUC = []

  for Depth in Profundidades:
    for n_estimators in Estimadores:
      lista_AUC_provisional = []

      for date in Fechas:
        Train_data = data_gym[data_gym['Mes'] <= date]
        Fecha_test = pd.Timestamp(date) + pd.DateOffset(months=1)
        Test_data = data_gym[data_gym['Mes'] == Fecha_test]
        
        X_train = Train_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
        y_train = Train_data['Baja']
        X_test = Test_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
        y_test = Test_data['Baja']    
    
        Modelo_random_forest = RandomForestClassifier(max_depth=Depth, n_estimators=n_estimators, random_state=1).fit(X_train, y_train)
        Predicciones_rndm_frst = Modelo_random_forest.predict_proba(X_test)
        AUC_PR = average_precision_score(y_test, Predicciones_rndm_frst[:, 1])
        lista_AUC_provisional.append(AUC_PR)

      Mean_AUC = np.mean(lista_AUC_provisional)
      lista_AUC.append(Mean_AUC)    
      lista_parametros.append([Depth, n_estimators])

  AUC_max_value = np.argmax(lista_AUC)
  print(f'El AUC del modelo Random Forest es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Gradient Boosting Classifier

In [9]:
def Gradient_Boosting (Fechas, Profundidades, Estimadores):

    lista_parametros = []
    lista_AUC = []

    for Depth in Profundidades:
        for n_estimators in Estimadores:
            lista_AUC_provisional = []
        
            for date in Fechas:
                Train_data = data_gym[data_gym['Mes'] <= date]
                Fecha_test = pd.Timestamp(date) + pd.DateOffset(months=1)
                Test_data = data_gym[data_gym['Mes'] == Fecha_test]
        
                X_train = Train_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
                y_train = Train_data['Baja']
                X_test = Test_data.drop(columns = ['IdPersona', 'Baja', 'Mes'])
                y_test = Test_data['Baja']

                Modelo_xgboost = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=Depth, objective='binary:logistic').fit(X_train, y_train)
                Predicciones_xgb = Modelo_xgboost.predict_proba(X_test)
                AUC_PR = average_precision_score(y_test, Predicciones_xgb[:, 1])
                lista_AUC_provisional.append(AUC_PR)

            Mean_AUC = np.mean(lista_AUC_provisional)
            lista_AUC.append(Mean_AUC)
            lista_parametros.append([Depth, n_estimators])

    AUC_max_value = np.argmax(lista_AUC)
    print(f'El AUC del modelo XGBoost es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Variables de los Modelos

In [16]:
Fechas = sorted(data_gym['Mes'].unique())
Fechas.pop()
Profundidades = [2, 3, 4, 5, 6, 7, 8, 9, 10]
Estimadores = [20, 40, 60, 80, 100, 120, 140]

# Ejecución de Modelos

In [11]:
Logistic_Regression(Fechas)

El AUC del Modelo de Regresión Logística es: 0.1867188689791308


In [17]:
Gradient_Boosting(Fechas, Profundidades, Estimadores)

El AUC del modelo XGBoost es 0.23048417754611003, para un nivel de profundidad de 2 y consta de 60 árboles de decisión


In [13]:
Random_Forest(Fechas, Profundidades, Estimadores)

El AUC del modelo Random Forest es 0.22933269219424884, para un nivel de profundidad de 8 y consta de 100 árboles de decisión
